## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [30]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-04-25 15:00:44--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.3’

        concrete_da   2%[                    ]   5.18M  12.9MB/s               ^C


In [ ]:
!unzip concrete_data_week4.zip

Archive:  concrete_data_week4.zip
replace concrete_data_week4/valid/positive/16679_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

Import libraries 

In [ ]:
import keras 

from keras.applications import VGG16

from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model

Define values of variables 

In [ ]:
batch_size_validation = 100
batch_size_training = 100

image_resize = 224
num_classes = 2 

Define image generator to incerase image data 

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input, 
)

Create generator for traning data 

In [ ]:
training_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Create generator for validation data 

In [ ]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Define Sequantial model to create CNN 

In [ ]:
model = Sequential()

Add pre-trained VGG16 model to our model and to adapt our input data drop head of the model 

In [ ]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Append output layer to model with # of classes 

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

Let's see what looks like our model

In [ ]:
model.layers

Let's check VGG16 models layers

In [ ]:
model.layers[0].layers

VGG16 is already trained , so we make that layers untrainable.

In [ ]:
model.layers[0].trainable = False

Let's see what looks like our model summary 

In [ ]:
model.summary()

Configure our model algorithm for compile 

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Set steps and epoch values 

In [ ]:
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Train our model 

In [ ]:
fit_history = model.fit_generator(
    training_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Save our model to use again without train in the future 

In [ ]:
model.save('classifier_vgg16.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

Load already saved model from directroy 

In [ ]:
model_resnet = load_model('classifier_resnet_model.h5')
model_vgg16 = load_model('classifier_vgg16.h5')

Create generator for test data 

In [ ]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False
    )

Evaluate test data with model and get results 

In [ ]:
score_vgg16 = model.evaluate_generator(test_generator, steps=None, verbose=1)
score_resnet = model_resnet.evaluate_generator(test_generator, steps=None, verbose=1)

Let's see what is our performance for each model 

In [ ]:
print('VGG16 score       : ', score_vgg16[0])  
print('VGG16 accuracy    : ', score_vgg16[1])  
print('ResNet50 score    : ', score_resnet[0])  
print('ResNet50 accuracy : ', score_resnet[1])  

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

Predict results for test images

In [ ]:
vgg16_predicts  = model_vgg16.predict_generator(test_generator, verbose=0)

resnet_predicts  = model_resnet.predict_generator(test_generator, verbose=0)

In [ ]:
print('Prediction for model VGG16:')
for i in vgg16_predicts[:5] : 
    if i[1] > i[0] : 
        print('Positive')
    else : 
        print('Negative')

In [ ]:
print('Prediction for model ResNet50:')
for i in resnet_predicts[:5] : 
    if i[1] > i[0] : 
        print('Positive')
    else : 
        print('Negative')